<a href="https://colab.research.google.com/github/himanshukoul/FIFA-PREDICTION-TASK2/blob/main/FIFA_Pred_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [117]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("international_matches.csv")
#from 2016
y = dataset.iloc[18318:, 16].values

#keeping away stats together
dataset.insert(24, "away_team_goalkeeper_score",dataset.pop("away_team_goalkeeper_score"))

same_country_as_home = []
for index, row in dataset.iterrows():
    if row['country'] == row['home_team']:
        same_country_as_home.append(0)
    elif row['country'] == row['away_team']:
        same_country_as_home.append(1)
    else:
      same_country_as_home.append(2)

dataset.insert(1, "same_country_as_home", same_country_as_home)
print(dataset.columns)
print(dataset.isna().sum())
#filling nan
for i in range(-1,-5,-1):
  homeTstats = dataset.columns[i-4]
  dataset[homeTstats] = dataset[homeTstats].fillna(dataset.groupby('home_team')[homeTstats].transform('mean'))
  awayTstats = dataset.columns[i]
  dataset[awayTstats] = dataset[awayTstats].fillna(dataset.groupby('away_team')[awayTstats].transform('mean'))
x = dataset.iloc[18318:,:].values
print(x[0])
print(y[0])
dataset = dataset.drop(dataset.columns[[0,2,3,4,5,8,9,10,11,13,14,16,17]],axis=1)
print(dataset.columns)
print(dataset.isna().sum())
x = dataset.iloc[18318:,:].values
print(x[0])
print(y[0])
#some stats are still nan due to no stats available by country
x_num = x[:,-8:].astype(float)
mask = ~np.isnan(x_num).any(axis=1)
print(mask)
x = x[mask]
y = y[mask]


Index(['date', 'same_country_as_home', 'home_team', 'away_team',
       'home_team_continent', 'away_team_continent', 'home_team_fifa_rank',
       'away_team_fifa_rank', 'home_team_total_fifa_points',
       'away_team_total_fifa_points', 'home_team_score', 'away_team_score',
       'tournament', 'city', 'country', 'neutral_location', 'shoot_out',
       'home_team_result', 'home_team_goalkeeper_score',
       'home_team_mean_defense_score', 'home_team_mean_offense_score',
       'home_team_mean_midfield_score', 'away_team_mean_defense_score',
       'away_team_mean_offense_score', 'away_team_mean_midfield_score',
       'away_team_goalkeeper_score'],
      dtype='object')
date                                 0
same_country_as_home                 0
home_team                            0
away_team                            0
home_team_continent                  0
away_team_continent                  0
home_team_fifa_rank                  0
away_team_fifa_rank                  0
home_

In [118]:
print(np.isnan(x[:,-8:].astype(float)).any())
print(len(x))
print(x[0])
print(y[0])

False
2572
[2 35 93 'Friendly' True 73.0 77.0 80.7 76.8 66.66170212765957
 61.817857142857136 63.56 64.0]
Draw


In [18]:
"""#filling leftover nans
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer.fit(x[:, -8:])
x[:, -8:] = imputer.transform(x[:, -8:])"""


In [119]:
print(x[0])
print(len(np.unique(x[:,0])))
print(len(np.unique(x[:,3])))
print(len(np.unique(x[:,4])))

[2 35 93 'Friendly' True 73.0 77.0 80.7 76.8 66.66170212765957
 61.817857142857136 63.56 64.0]
2
29
2


In [24]:
#!pip install category_encoders

In [120]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
#import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
print(y)
y = le.fit_transform(y)
print(y)

#ct =  ColumnTransformer(transformers=[('encoder', ce.BinaryEncoder(), [0,1,5]),('encoder2', ce.BinaryEncoder(), [4])], remainder='passthrough')
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), [0,3,4],)], remainder='passthrough')
x = np.array(ct.fit_transform(x))

print(x[0])



['Draw' 'Lose' 'Win' ... 'Lose' 'Lose' 'Win']
[0 1 2 ... 1 1 2]
[0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 35 93 73.0
 77.0 80.7 76.8 66.66170212765957 61.817857142857136 63.56 64.0]


In [121]:
print(len(x[0]))

43


In [122]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [123]:
print(x_train[0])

[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 50 13 75.0
 74.0 76.3 80.0 85.2 82.3 84.0 81.0]


In [124]:
from sklearn.preprocessing import StandardScaler
#29+2+2 = 33 encoded
sc = StandardScaler()
sc.fit(x_train[:,33:])
x_train[:, 33:] = sc.transform(x_train[:, 33:])
x_test[:, 33:] = sc.transform(x_test[:, 33:])
print(x_train[0])


[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0
 0.1999257156088542 -0.9727096850708493 -0.18613471788009436
 -0.2782756399585211 0.03437073043602495 0.5743701557416129
 1.8466776818206703 1.1536946461999542 1.4072706769493415
 0.7472928132485457]


In [125]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'minkowski', p = 2)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)   #49%

[[  9  43  75]
 [ 24  63  56]
 [ 28  35 182]]


0.49320388349514566

In [126]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'sigmoid', random_state = 0)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)  #43.8%

[[ 35  51  41]
 [ 42  75  26]
 [ 70  59 116]]


0.4388349514563107

In [127]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)      # 47

[[ 34  40  53]
 [ 35  64  44]
 [ 53  44 148]]


0.47766990291262135

In [128]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 30, criterion = 'log_loss', random_state = 0)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)         # 49.12

[[ 13  41  73]
 [ 29  66  48]
 [ 38  33 174]]


0.4912621359223301

In [129]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)          #59.6% (came best)

[[  4  40  83]
 [  0  90  53]
 [  0  32 213]]


0.596116504854369

In [130]:
print("give a list with params for football match")
#[2 35 93 'Friendly' True 73.0 77.0 80.7 76.8 66.66170212765957 61.817857142857136 63.56 64.0]
#eg ['England','Spain',7,5,'FIFA World Cup qualification','England',False,83,84,88,82,86,86,81,80]
x_input = eval(input())
if(x_input[0]==x_input[5]):
  x_input[0] = 0
elif(x_input[1]==x_input[5]):
  x_input[0] = 1
else:
  x_input[0] = 2
x_input.pop(1)
x_input.pop(4)
print(x_input)
x_input = np.array([x_input], dtype=object)
print(x_input)
x_input = ct.transform(x_input)
x_input[:,33:] = sc.transform(x_input[:,33:])

y_pred = classifier.predict(x_input)
if(y_pred==0):
  print("draw")
elif(y_pred==1):
  print("1st team lose")
else:
  print("1st team win")


give a list with params for football match
['England','Spain',7,5,'FIFA World Cup qualification','England',False,83,84,88,82,86,86,81,80]
[0, 7, 5, 'FIFA World Cup qualification', False, 83, 84, 88, 82, 86, 86, 81, 80]
[[0 7 5 'FIFA World Cup qualification' False 83 84 88 82 86 86 81 80]]
1st team win
